# Blizzard Commodities DataFrame (79+M rows)

## Data pulled hourly between 2023-12-22 19:41:00 and 2024-01-03 12:41:00

## Imports Section

In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import numpy as np
import statistics
from datetime import datetime
from random import sample

## Open csv / Load to DF

In [2]:
start=datetime.now()
display(f"({start}) Loading csv file ...")
Blizzard_Commodities_DF=pd.read_csv('commodities_data.csv')
Blizzard_Commodities_DF.shape
display(f"(Time to load : {datetime.now()-start}")

'(2024-01-08 18:02:05.982118) Loading csv file ...'

'(Time to load : 0:00:57.655398'

## Find na/NaN values

In [ ]:
Blizzard_Commodities_DF.isna().sum()

## DF Structure (Review/Correct)

In [3]:
Blizzard_Commodities_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79852830 entries, 0 to 79852829
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          int64 
 1   quantity    int64 
 2   unit_price  int64 
 3   time_left   object
 4   item.id     int64 
 5   item.name   object
 6   item.class  object
 7   datetime    object
dtypes: int64(4), object(4)
memory usage: 4.8+ GB


In [4]:
start=datetime.now()

In [5]:
# Change datedime columns to datetime
Blizzard_Commodities_DF['datetime'] = pd.to_datetime(Blizzard_Commodities_DF['datetime'])

In [6]:
# Change id columns to string
display(f"({start}) Re-Casting id columns ...")
Blizzard_Commodities_DF['id']=Blizzard_Commodities_DF['id'].astype(str)
Blizzard_Commodities_DF['item.id']=Blizzard_Commodities_DF['item.id'].astype(str)
display(f"(Time to cast : {datetime.now()-start}")

'(2024-01-08 18:03:03.658390) Re-Casting id columns ...'

'(Time to cast : 0:00:47.045537'

In [7]:
# Verifiy change
Blizzard_Commodities_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79852830 entries, 0 to 79852829
Data columns (total 8 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   quantity    int64         
 2   unit_price  int64         
 3   time_left   object        
 4   item.id     object        
 5   item.name   object        
 6   item.class  object        
 7   datetime    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 4.8+ GB


## Data Overview

In [ ]:
Blizzard_Commodities_DF

### Initial Value Analisys by Column

- id

In [ ]:
Blizzard_Commodities_DF['id'].value_counts()

- id .. from this we can tell auctions last no more than 50 hrs, a bit over 2 days.

In [ ]:
Blizzard_Commodities_DF['id'].value_counts().plot()

In [8]:
auction_id_cc=pd.DataFrame(Blizzard_Commodities_DF['id'].value_counts().value_counts())

In [9]:
auction_id_cc

,count
count,
1,2109005
2,930071
12,612889
48,468105
24,463246
3,364507
4,231023
5,172418
6,138057


In [10]:
auction_id_cc.rename(columns={'count':'repetitions'}, inplace=True)

In [11]:
auction_id_cc.reset_index(inplace=True)

In [12]:
auction_id_cc.sort_values('count', inplace=True)

In [13]:
auction_id_cc

,count,repetitions
0,1,2109005
1,2,930071
5,3,364507
6,4,231023
7,5,172418
8,6,138057
10,7,119175
12,8,103680
13,9,95134
14,10,87794


In [ ]:
auction_id_cc['repetitions'].plot()

In [34]:
ax=0

In [36]:
x=auction_id_cc['count']
y=auction_id_cc['repetitions']
plt.scatter(x,y)
plt.xticks(rotation='vertical')
plt.show()

TypeError: 'tuple' object is not callable

In [ ]:
auction_id_cc.reset_index(drop=True, inplace=True)

In [ ]:
Blizzard_Commodities_DF['id'].value_counts().value_counts().plot()

- quantity

In [ ]:
Blizzard_Commodities_DF['quantity'].value_counts()

In [ ]:
Blizzard_Commodities_DF['quantity'].value_counts().plot()

- unit_price

In [ ]:
Blizzard_Commodities_DF['unit_price'].value_counts()

In [ ]:
Blizzard_Commodities_DF['unit_price'].value_counts().plot()

- time_left

In [ ]:
Blizzard_Commodities_DF['time_left'].value_counts()

In [ ]:
Blizzard_Commodities_DF['time_left'].value_counts().plot()

- item.id

- item.name

In [ ]:
Blizzard_Commodities_DF['item.name'].value_counts() 

In [ ]:
Blizzard_Commodities_DF['item.name'].value_counts().plot()

- item.class

In [ ]:
Blizzard_Commodities_DF['item.class'].value_counts()

In [ ]:
Blizzard_Commodities_DF['item.class'].value_counts().plot()

- date_time

In [ ]:
Blizzard_Commodities_DF['datetime'].value_counts()

In [ ]:
Blizzard_Commodities_DF['datetime'].value_counts().plot()

### ** Modify DF for easier analysis of datetime data

In [ ]:
Blizzard_Commodities_DF['date']=pd.to_datetime(Blizzard_Commodities_DF['datetime']).dt.date
Blizzard_Commodities_DF['date']=pd.to_datetime(Blizzard_Commodities_DF['date'])
Blizzard_Commodities_DF['time']=pd.to_datetime(Blizzard_Commodities_DF['datetime']).dt.time
# Blizzard_Commodities_DF.drop(columns='datetime', inplace=True)

In [ ]:
Blizzard_Commodities_DF.info()

## Divide Blizzard_Commodities_DF in Blizzard_1 and Blizzard_2 

## and save as CSV to close the DF and use less memory

In [ ]:
Blizzard_1_df=Blizzard_Commodities_DF.iloc[0:int(79852830/2)]
Blizzard_1_df.to_csv('Blizzard_1.csv')
Blizzard_2_df=Blizzard_Commodities_DF.iloc[int(79852830/2):]
Blizzard_2_df.to_csv('Blizzard_2.csv')
Blizzard_Commodities_DF=[]
Blizzard_2_df=[]

- date

In [ ]:
Blizzard_Commodities_DF['date'].value_counts()

In [ ]:
Blizzard_Commodities_DF['date'].value_counts().plot()

- time

In [ ]:
Blizzard_Commodities_DF['time'].value_counts()

In [ ]:
Blizzard_Commodities_DF['time'].value_counts().plot()

## Analysis based on 'item.name' column

In [ ]:
item_name_count_df=pd.DataFrame(Blizzard_Commodities_DF['item.name'].value_counts())
item_name_count_df.reset_index()

### The most offered item is 'Fluorescent Fluid', analysing data

In [ ]:
fluorecent_fluid_df=Blizzard_Commodities_DF.loc[Blizzard_Commodities_DF['item.name']=='Fluorescent Fluid']

In [ ]:
#Did not work as I expected
for column in fluorecent_fluid_df:
    try:
        display(f"Columns: {column}")
        display(fluorecent_fluid_df[column].value_counts())
    except Exception as err:
        print(err)
    try:
        display(fluorecent_fluid_df[column].value_counts().plot())
    except Exception as err:
        print(err)

In [ ]:
fluorecent_fluid_df.info()

In [ ]:
fluorecent_fluid_df['id'].value_counts()

In [ ]:
fluorecent_fluid_df['id'].value_counts().plot()

In [ ]:
fluorecent_fluid_df['quantity'].value_counts()

In [ ]:
fluorecent_fluid_df['quantity'].value_counts().plot()

In [ ]:
fluorecent_fluid_df['unit_price'].value_counts()

In [ ]:
fluorecent_fluid_df['unit_price'].value_counts().plot()

In [ ]:
fluorecent_fluid_df['time_left'].value_counts()

In [ ]:
fluorecent_fluid_df['time_left'].value_counts().plot()

In [ ]:
fluorecent_fluid_df['date'].value_counts()

In [ ]:
fluorecent_fluid_df['date'].value_counts().plot()

In [ ]:
fluorecent_fluid_df['time'].value_counts()

In [ ]:
fluorecent_fluid_df['time'].value_counts().plot()

In [ ]:
time_plot=pd.DataFrame(fluorecent_fluid_df['time'].value_counts())
time_plot.info()

In [ ]:
time_plot

In [ ]:
xtick=[str(d) for d in time_plot.index]
y=time_plot['count']
plt.plot()

In [ ]:
time_plot.sort_values(0,inplace=True)
time_plot.reset_index(drop=True)
time_plot[0]=time_plot[0].astype(str)

In [ ]:
time_plot

In [ ]:
y.info()

In [ ]:
time_plot.plot()

### Find the auction(s) the lasted the longest

### (Waste of 2 hrs to process, same results as 20 sec value_counts())

In [ ]:
# get the auction(s) that was live the longest
# max=0
# id_max=0
# max_ids=[]
# for id in fluorecent_fluid_df['id'].unique():
#     display(id)
#     days=fluorecent_fluid_df['date'].loc[fluorecent_fluid_df['id']==id]
#     if len(list(set(days)))>max:
#         max=len(list(set(days)))
#         id_max=id
#     if len(list(set(days)))==max:
#         max_ids.append(id)
#         display(f"id:{id} max with :{max}")
# display(id_max, max)

In [ ]:
fluorecent_fluid_df.sort_values(['date','time'], ascending=[True,True])
fluorecent_fluid_df.head(30)

In [ ]:
fluorecent_fluid_df.tail()

In [ ]:
fluorecent_fluid_df.loc[fluorecent_fluid_df['id']==max_ids[0]].shape

In [ ]:
from random import sample
list=sample(max_ids,int(len(max_ids)/100))
for i in list:
    print(i, fluorecent_fluid_df.loc[fluorecent_fluid_df['id']==i].shape)


In [ ]:
Exceptional_Pelt_df=Blizzard_Commodities_DF.loc[Blizzard_Commodities_DF['item.name']=='Exceptional Pelt']

In [ ]:
Exceptional_Pelt_df.to_csv('Exceptional_Pelt.csv',index=False)

## Group DF by item.name

In [ ]:
Blizzard_Commodities_DF.info()

In [ ]:
now=datetime.now()
# grouped_DF=Blizzard_Commodities_DF['id','quantity','unit_price','item.name'].groupby('item.name').sum()
grouped_DF=Blizzard_Commodities_DF[['id','quantity','unit_price','item.name']]
print(F"Time elapsed {datetime.now()-now}")

In [ ]:
grouped_DF

In [ ]:
start=datetime.now()
print(f"({start}) Export item.name and (price, quantity) columns ...")

In [ ]:
# grouped_DF=Blizzard_Commodities_DF[['id','quantity','unit_price','item.name']].groupby('item.name').sum()
name_price_df=Blizzard_Commodities_DF[['item.name','unit_price']]
name_quantity_df=Blizzard_Commodities_DF[['item.name','quantity']]
print(F"Time elapsed {datetime.now()-start}")

In [ ]:
start=datetime.now()
print(f"({start}) Groupping by item.name and summing quantity and unit_price columns ...")

In [ ]:
datetime.now()
grouped_DF

In [ ]:
start=datetime.now()
print(f"({start}) Grupping by item.name and summing quantity and unit_price columns ...")

In [ ]:
name_price_df.sum()

In [ ]:
import gc

In [ ]:
del Blizzard_Commodities_DF

In [ ]:
gc.collect()

In [ ]:
blizzard_1_df=pd.read_csv('blizzard_2.csv')
blizzard_1_df

In [ ]:
blizzard_1_df.columns

In [ ]:
blizzard_1_df.drop(columns=['Unnamed: 0', 'time_left', 'item.id',
       'id', 'item.class', 'datetime', 'date', 'time'],axis=1,inplace=True)

In [ ]:
blizzard_1_df.columns

In [ ]:
now=datetime.now()
now

In [ ]:
numeric_columns_df=Blizzard_Commodities_DF[['quantity', 'unit_price', 'item.name']]

In [ ]:
groupby_df=numeric_columns_df.groupby('item.name').sum()

In [ ]:
numeric_columns_df

In [ ]:
groupby_df

In [ ]:
groupby_df['average_price']=groupby_df['unit_price']/groupby_df['quantity']

In [ ]:
groupby_df['price_mode']=statistics.mode(groupby_df['unit_price'])

In [ ]:
groupby_df.iloc[0]

In [ ]:
groupby_df.reset_index(inplace=True)

In [ ]:
groupby_df.sort_values('unit_price')